In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import sqlite3
import numpy as np

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

from sklearn.model_selection import KFold
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [4]:
path1 = "/content/drive/MyDrive/DS340W_OilFutures/Data/WTI_Lag_Train.csv"
path2 = "/content/drive/MyDrive/DS340W_OilFutures/Data/WTI_Lag_Val.csv"
path3 = "/content/drive/MyDrive/DS340W_OilFutures/Data/WTI_Lag_Test.csv"

WTI_Lag_Train = pd.read_csv(path1)
WTI_Lag_Val = pd.read_csv(path2)
WTI_Lag_Test = pd.read_csv(path3)

path4 = "/content/drive/MyDrive/DS340W_OilFutures/Data/WTI 2000-2026.csv"
df = pd.read_csv(path4)

In [5]:
df

,Date,Price
0,2000-01-04,25.55
1,2000-01-05,24.91
2,2000-01-06,24.78
3,2000-01-07,24.22
4,2000-01-10,24.67
...,...,...
6678,2026-01-15,59.08
6679,2026-01-16,59.34
6680,2026-01-18,58.97
6681,2026-01-19,59.39


In [6]:
df = df.copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

## Build supervised train/test dfs for benchmark models (RF, SVR, XGBoost)

In [7]:
def make_lag_supervised(df, n_lags=10, horizon=1):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    for k in range(1, n_lags + 1):
        df[f"Price_lag{k}"] = df["Price"].shift(k)

    df["y"] = df["Price"].shift(-horizon)  # next-day target
    df = df.dropna().reset_index(drop=True)

    X = df[[f"Price_lag{k}" for k in range(1, n_lags + 1)]].to_numpy()
    y = df["y"].to_numpy()
    dates = df["Date"].to_numpy()
    return X, y, dates

X, y, dates = make_lag_supervised(df, n_lags=10, horizon=1)

# 90/10 time split
n = len(y)
split = int(0.9 * n)

X_train, y_train = X[:split], y[:split]
X_test,  y_test  = X[split:], y[split:]
dates_test = dates[split:]

print("Train:", dates[0], "→", dates[split-1])
print("Test :", dates[split], "→", dates[-1])

Train: 2000-01-19T00:00:00.000000000 → 2023-06-22T00:00:00.000000000
Test : 2023-06-23T00:00:00.000000000 → 2026-01-19T00:00:00.000000000


## Define Evaluation Metrics

In [8]:
def regression_metrics(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)

    # Relative RMSE (normalized by mean of true values)
    rrmse = rmse / np.mean(y_true)

    # Pearson correlation coefficient
    cc = np.corrcoef(y_true, y_pred)[0, 1]

    return {
        "RMSE": rmse,
        "RRMSE": rrmse,
        "MAE": mae,
        "CC": cc
    }

# Random Forest

In [9]:
rf = RandomForestRegressor(
    n_estimators=500,
    random_state=42,
    n_jobs=-1,
    max_features="sqrt"
)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

rf_metrics = regression_metrics(y_test, rf_pred)

# Support Vector Regrssion (RBF Kernel)

In [10]:
svr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", SVR(kernel="rbf", C=10.0, epsilon=0.1, gamma="scale"))
])

svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)

svr_metrics = regression_metrics(y_test, svr_pred)

# Extreme Gradient Boosting (XGBoost)

In [11]:
xgb = XGBRegressor(
    n_estimators=1500,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

xgb_metrics = regression_metrics(y_test, xgb_pred)

# Benchmark Model Metrics

In [12]:
results_table = pd.DataFrame.from_dict({
    "Random Forest": rf_metrics,
    "SVR": svr_metrics,
    "XGBoost": xgb_metrics,
}, orient="index")

results_table

,RMSE,RRMSE,MAE,CC
Random Forest,2.055620,0.028687,1.535970,0.968338
SVR,1.906503,0.026606,1.448466,0.972991
XGBoost,2.007251,0.028012,1.516466,0.969915


## Paper-style 10-model CV ensemble for Gaussian Process Regression (GPR)

In [13]:
N_LAGS = 10

for k in range(1, N_LAGS + 1):
    df[f"Price_lag{k}"] = df["Price"].shift(k)

# Drop rows that don't have full lag history
df_lag = df.dropna().reset_index(drop=True)

df_lag.head()

,Date,Price,Price_lag1,Price_lag2,Price_lag3,Price_lag4,Price_lag5,Price_lag6,Price_lag7,Price_lag8,Price_lag9,Price_lag10
0,2000-01-19,29.54,28.85,28.02,26.69,26.28,25.77,24.67,24.22,24.78,24.91,25.55
1,2000-01-20,29.66,29.54,28.85,28.02,26.69,26.28,25.77,24.67,24.22,24.78,24.91
2,2000-01-21,28.20,29.66,29.54,28.85,28.02,26.69,26.28,25.77,24.67,24.22,24.78
3,2000-01-24,27.83,28.20,29.66,29.54,28.85,28.02,26.69,26.28,25.77,24.67,24.22
4,2000-01-25,28.28,27.83,28.20,29.66,29.54,28.85,28.02,26.69,26.28,25.77,24.67


In [14]:
df_lag["y"] = df_lag["Price"]

# Drop last row(s) where target is missing
df_lag = df_lag.dropna().reset_index(drop=True)


In [15]:
lag_cols = [f"Price_lag{k}" for k in range(1, N_LAGS + 1)]

X = df_lag[lag_cols].astype(float).to_numpy()
y = df_lag["y"].astype(float).to_numpy()
dates = df_lag["Date"].to_numpy()

In [16]:
n_total = len(df_lag)
train_size = int(0.9 * n_total)

X_train = X[:train_size]
y_train = y[:train_size]
dates_train = dates[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]
dates_test = dates[train_size:]

print("Train period:", dates_train[0], "→", dates_train[-1])
print("Test period :", dates_test[0],  "→", dates_test[-1])

Train period: 2000-01-19T00:00:00.000000000 → 2023-06-23T00:00:00.000000000
Test period : 2023-06-26T00:00:00.000000000 → 2026-01-20T00:00:00.000000000


## Kernel: isotropic squared exponential (paper Eq. 2)

isotropic (single length scale), squared exponential (RBF), σ²ₙ noise term (required by real financial)

In [18]:
kernel = (
    ConstantKernel(1.0, (1e-1, 1e4)) *
    RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e4))
    + WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-1, 1e3))
)

## 10-model CV ensemble
Research performs 10-fold CV inside training set and averages predictions.

In [ ]:
from tqdm.auto import tqdm

kf = KFold(n_splits=5, shuffle=True, random_state=42)

preds = []
stds  = []

for fold, (tr_idx, va_idx) in enumerate(
        tqdm(kf.split(X_train), total=5, desc="10-Fold GPR"), start=1):

    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_train[tr_idx])
    y_tr = y_train[tr_idx]

    X_te = scaler.transform(X_test)

    gpr = GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=False,
        n_restarts_optimizer=0,
        random_state=42
    )

    gpr.fit(X_tr, y_tr)
    yhat, ystd = gpr.predict(X_te, return_std=True)

    preds.append(yhat)
    stds.append(ystd)

10-Fold GPR:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

preds = []
stds  = []

for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train), start=1):
    # Standardize predictors (paper explicitly says standardized predictors)
    scaler = StandardScaler()
    X_tr = scaler.fit_transform(X_train[tr_idx])
    y_tr = y_train[tr_idx]

    X_te = scaler.transform(X_test)

    gpr = GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=False,        # "empty basis function" style
        n_restarts_optimizer=5,
        random_state=42
    )

    gpr.fit(X_tr, y_tr)
    yhat, ystd = gpr.predict(X_te, return_std=True)

    preds.append(yhat)
    stds.append(ystd)

    print(f"CV{fold} kernel:", gpr.kernel_)

/usr/local/lib/python3.12/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


CV1 kernel: 31.6**2 * RBF(length_scale=2.16) + WhiteKernel(noise_level=2.05)


KeyboardInterrupt: 

In [ ]:
y_pred = np.mean(np.vstack(preds), axis=0)
y_std  = np.mean(np.vstack(stds), axis=0)

results = pd.DataFrame({
    "Date": dates_test,
    "y_true": y_test,
    "y_pred": y_pred,
    "y_std": y_std
}).sort_values("Date")

results.head()

break

In [24]:
def prepare_xy(df, n_lags=10, horizon=0):
    """
    horizon=0 -> predict today's Price from lags (super common)
    horizon=1 -> predict next-day Price (paper-style 1-day ahead)
    """
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").reset_index(drop=True)

    lag_cols = [f"Price_lag{i}" for i in range(1, n_lags + 1)]
    X = df[lag_cols].astype(float)

    if horizon == 0:
        y = df["Price"].astype(float)
        dates = df["Date"]
    else:
        # next-day target
        y = df["Price"].shift(-horizon).astype(float)
        dates = df["Date"]
        # drop last horizon rows because y becomes NaN
        X = X.iloc[:-horizon].copy()
        y = y.iloc[:-horizon].copy()
        dates = dates.iloc[:-horizon].copy()

    # remove any rows with missing values (e.g., if you have "--" or NaNs)
    mask = X.notna().all(axis=1) & y.notna()
    return X.loc[mask].to_numpy(), y.loc[mask].to_numpy(), dates.loc[mask].to_numpy(), lag_cols


def gpr_cv_ensemble_predict(train_df, eval_df, n_lags=10, horizon=1, n_splits=10, random_state=42):
    # prepare train/eval
    X_train, y_train, train_dates, lag_cols = prepare_xy(train_df, n_lags=n_lags, horizon=horizon)
    X_eval,  y_eval,  eval_dates,  _       = prepare_xy(eval_df,  n_lags=n_lags, horizon=horizon)

    d = X_train.shape[1]  # number of predictors

    # Eq (2): isotropic squared exponential = ConstantKernel * RBF(scalar length_scale)
    kernel = (
        ConstantKernel(1.0, (1e-3, 1e3)) *
        RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3))
        + WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-8, 1e1))
    )

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    preds = []
    stds  = []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train), start=1):
        # Standardize predictors (paper says standardized predictors)
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_train[tr_idx])
        y_tr = y_train[tr_idx]

        X_ev = scaler.transform(X_eval)

        gpr = GaussianProcessRegressor(
            kernel=kernel,
            normalize_y=False,       # "empty basis" vibe; keep mean handling minimal
            n_restarts_optimizer=5,  # helps hyperparameter estimation
            random_state=random_state
        )

        gpr.fit(X_tr, y_tr)
        yhat, ystd = gpr.predict(X_ev, return_std=True)

        preds.append(yhat)
        stds.append(ystd)

        print(f"Fold {fold} fit kernel:", gpr.kernel_)

    # average across CV1..CV10
    y_pred = np.mean(np.vstack(preds), axis=0)
    y_std  = np.mean(np.vstack(stds), axis=0)

    out = pd.DataFrame({
        "Date": eval_dates,
        "y_true": y_eval,
        "y_pred": y_pred,
        "y_std": y_std
    }).sort_values("Date").reset_index(drop=True)

    return out

In [25]:
pred_val  = gpr_cv_ensemble_predict(WTI_Lag_Train, WTI_Lag_Val,  n_lags=10, horizon=1)
pred_test = gpr_cv_ensemble_predict(WTI_Lag_Train, WTI_Lag_Test, n_lags=10, horizon=1)

pred_test.head()

NameError: name 'ConstantKernel' is not defined